In [41]:
from pulp import *
import pandas as pd

def maximize_vorp(players, roster_size, min_qbs, max_qbs, min_rbs, max_rbs, min_wrs, max_wrs, min_tes, max_tes,
                  starting_qb, starting_rb, starting_wr, starting_te, starting_flex, picks, taken, adp):
    # Initialize model
    model = LpProblem("FantasyFootballOptimization", LpMaximize)
    
    # Decision Variables: Binary variables indicating if a player is selected and/or starting
    players_selected = LpVariable.dicts("PlayerSelected", players, cat='Binary')
    players_starting = LpVariable.dicts("PlayerStarting", players, cat='Binary')
    
    # Objective Function: Maximize total VORP, with a bonus for starting players
    model += lpSum([vorp[player] * players_selected[player] for player in players]) \
            + lpSum([vorp[player] * 2 * players_starting[player] for player in players]), "TotalVORP"
    
    # Constraints
    # Roster size constraint
    model += lpSum([players_selected[player] for player in players]) == roster_size
    
    # Positional constraints: Minimum and maximum number of players at each position
    model += lpSum([players_selected[player] for player in players if player_position[player] == "QB"]) >= min_qbs
    model += lpSum([players_selected[player] for player in players if player_position[player] == "QB"]) <= max_qbs
    model += lpSum([players_selected[player] for player in players if player_position[player] == "RB"]) >= min_rbs
    model += lpSum([players_selected[player] for player in players if player_position[player] == "RB"]) <= max_rbs
    model += lpSum([players_selected[player] for player in players if player_position[player] == "WR"]) >= min_wrs
    model += lpSum([players_selected[player] for player in players if player_position[player] == "WR"]) <= max_wrs
    model += lpSum([players_selected[player] for player in players if player_position[player] == "TE"]) >= min_tes
    model += lpSum([players_selected[player] for player in players if player_position[player] == "TE"]) <= max_tes
    
    # Starting lineup constraints: Number of players at each position
    model += lpSum([players_starting[player] for player in players if player_position[player] == "QB"]) == starting_qb
    model += lpSum([players_starting[player] for player in players if player_position[player] == "RB"]) >= starting_rb
    model += lpSum([players_starting[player] for player in players if player_position[player] == "WR"]) >= starting_wr
    model += lpSum([players_starting[player] for player in players if player_position[player] == "TE"]) >= starting_te
    # Flex position constraint
    model += lpSum([players_starting[player] for player in players if player_position[player] in ["RB", "WR","TE"]]) == \
        starting_wr + starting_rb + starting_te + starting_flex

    # Constraint: Any player starting must also be selected
    for player in players:
        model += players_starting[player] <= players_selected[player]
    
    # Constraint: For each pick, we can only select players with an ADP greater than or equal to the pick and not already taken
    for pick in picks:
        available_players = [player for player in players if adp[player] >= pick and player not in taken]
        model += lpSum([players_selected[player] for player in available_players]) >= len(picks) - picks.index(pick)
        
    # Solve the problem
    model.solve()
    
    # Extract results
    selected_players = [player for player in players if players_selected[player].value() == 1]
    starting_players = [player for player in players if players_starting[player].value() == 1]
    total_vorp = value(model.objective)
    
    return selected_players, starting_players, total_vorp

if __name__ == "__main__":
    # Load data
    df = pd.read_csv('vorp.csv')
    # get into the right format
    df['Player'] = df['FirstName'] + ' ' + df['LastName']
    players = df['Player'].tolist()
    vorp = df.set_index('Player')['Y0_VORP'].to_dict()
    player_position = df.set_index('Player')['Position'].to_dict()
    ADP = df.set_index('Player')['RedraftHalfPPR'].to_dict()
    
    # Constraints
    min_qbs, max_qbs = 1, 3
    min_rbs, max_rbs = 2, 6
    min_wrs, max_wrs = 2, 6
    min_tes, max_tes = 1, 2
    starting_qb = 1
    starting_rb = 2
    starting_wr = 2
    starting_te = 1
    starting_flex = 1
    roster_size = 13
    
    taken = []
    teams = 12
    picks = []
    pick = 9
    for i in range(1,roster_size+1):
        for j in range(1,teams+1):
            if j == pick and i % 2 == 1:
                picks.append(((i-1)*teams)+j)
            elif j == teams - pick + 1 and i % 2 == 0:
                picks.append(((i-1)*teams)+j)
    
    # Constraints
    min_qbs, max_qbs = 1, 3
    min_rbs, max_rbs = 2, 6
    min_wrs, max_wrs = 2, 6
    min_tes, max_tes = 1, 2
    starting_qb = 1
    starting_rb = 2
    starting_wr = 2
    starting_te = 1
    starting_flex = 1
    
    # Run optimization
    selected_players, starting_players, total_vorp = maximize_vorp(players, roster_size, min_qbs, max_qbs, min_rbs, max_rbs,
                                                                   min_wrs, max_wrs, min_tes, max_tes, starting_qb, starting_rb,
                                                                   starting_wr, starting_te, starting_flex, picks, taken, ADP)
    
    # Output results, showing each pos, player, vorp, and adp sorted by adp
    selected_players = sorted(selected_players, key=lambda x: ADP[x])
    print('Selected Players')
    for player in selected_players:
        print(player, player_position[player], ADP[player])
        
    print('\nStarting Players')
    starting_players = sorted(starting_players, key=lambda x: ADP[x])
    for player in starting_players:
        print(player, player_position[player], ADP[player])
        


Welcome to the CBC MILP Solver 
Version: 2.10.3 
Build Date: Dec 15 2019 

command line - /Users/zachwayne/opt/anaconda3/lib/python3.9/site-packages/pulp/solverdir/cbc/osx/64/cbc /var/folders/b9/1w_dn_kj5zgffq25ztcmwj5m0000gn/T/1d6676c529784416b3411bbcc5d51035-pulp.mps max timeMode elapsed branch printingOptions all solution /var/folders/b9/1w_dn_kj5zgffq25ztcmwj5m0000gn/T/1d6676c529784416b3411bbcc5d51035-pulp.sol (default strategy 1)
At line 2 NAME          MODEL
At line 3 ROWS
At line 523 COLUMNS
At line 8172 RHS
At line 8691 BOUNDS
At line 9674 ENDATA
Problem MODEL has 518 rows, 982 columns and 5404 elements
Coin0008I MODEL read with 0 errors
Option for timeMode changed from cpu to elapsed
Continuous objective value is 133.467 - 0.00 seconds
Cgl0004I processed model has 514 rows, 982 columns (982 integer (982 of which binary)) and 4913 elements
Cbc0038I Initial state - 0 integers unsatisfied sum - 0
Cbc0038I Solution found of -133.467
Cbc0038I Before mini branch and bound, 982 integ